In [177]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from datetime import datetime

In [178]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import plotly.express as px
from sklearn.preprocessing import StandardScaler

In [179]:
abm = pd.read_csv("csv_files/abm.csv")
card = pd.read_csv("csv_files/card.csv")
cheque = pd.read_csv("csv_files/cheque.csv")
eft = pd.read_csv("csv_files/eft.csv")
emt = pd.read_csv("csv_files/emt.csv")
wire = pd.read_csv("csv_files/wire.csv")

kyc = pd.read_csv("csv_files/kyc.csv")
all_transactions = pd.read_csv("csv_files/all_transactions_month.csv")

1. Wire Transfers
Advantages: Ability to handle large money transfers, especially for international transactions.
Use: Due to its fast and international nature, wire transfers are a very popular method of money laundering, especially when it involves transferring funds to offshore accounts.

2. EMT (Email Transfer)
Advantages: Easy to operate, high popularity, usually used for smaller daily transactions.
Use: May be used for smaller money laundering operations, or as part of a "layering" strategy to obfuscate the flow of funds through frequent small transactions.

3. EFT (Electronic Funds Transfer)
Pros: Covers a variety of forms from direct deposit to automatic deductions and is suitable for domestic and international transactions.
Use: Similar to wire transfers, electronic funds transfers can be used for large-scale fund movements, especially when combined with multiple accounts and cross-border transactions.

4. ABM (Automated Banking Machine)
Advantages: Provides cash access services, users can anonymously conduct certain types of transactions.
Use: Although the transaction amount is limited, it can be used to deposit illegal cash and then make other forms of transfers.

5. Cheques (cheques)
Advantages: Traditional payment methods, used for both legal and illegal purposes have an extensive history.
Purpose: To launder funds through checks, especially by setting up shell companies to issue or receive checks.
Money laundering strategies:
Layering: Multiple transfers between the source and final destination of funds, using different methods and tools at each step.
Structuring: Make multiple small transactions to avoid triggering financial institutions' reporting thresholds.
Mixed use: By cross-using wire transfers, EFTs, checks, and other tools, money launderers try to make tracing more difficult.

EFT & Wire

EMT

ABM 

Cheque

# ABM


### 1. single transaction > 20k
### 2. single cash transaction > 10k

In [180]:
abm = abm.fillna('unknown')
abm.head(10)

,abm_id,customer_id,amount_cad,debit_credit,cash_indicator,country,province,city,transaction_date,transaction_time
0,ABM00000000000000006,SYNCID0000000014,25.41,credit,False,CA,SK,REGINA,2022-11-16,17:37:41
1,ABM00000000000000008,SYNCID0000000034,238.17,debit,True,CA,ON,TORONTO,2022-11-18,10:22:59
2,ABM00000000000000009,SYNCID0000000034,1655.43,credit,False,CA,unknown,other,2022-12-29,11:56:08
3,ABM00000000000000010,SYNCID0000000034,620.69,credit,True,CA,unknown,other,2023-01-22,16:48:12
4,ABM00000000000000011,SYNCID0000000034,323.70,debit,True,CA,unknown,other,2022-11-14,13:24:45
5,ABM00000000000000012,SYNCID0000000034,6876.76,credit,False,CA,ON,VAUGHAN,2022-12-11,11:55:50
6,ABM00000000000000013,SYNCID0000000038,104.55,debit,True,CA,unknown,other,2022-12-28,18:07:36
7,ABM00000000000000014,SYNCID0000000046,111.91,debit,True,CA,AB,EDMONTON,2022-11-14,22:58:02
8,ABM00000000000000015,SYNCID0000000046,193.76,debit,True,unknown,unknown,unknown,2022-11-26,10:50:41
9,ABM00000000000000016,SYNCID0000000046,217.00,debit,True,unknown,unknown,unknown,2023-01-06,10:26:31


In [181]:
# Strange Customer Example
abm[abm["customer_id"] == "SYNCID0000016882"]

,abm_id,customer_id,amount_cad,debit_credit,cash_indicator,country,province,city,transaction_date,transaction_time
20380,ABM00000000000021506,SYNCID0000016882,153.80,debit,True,unknown,unknown,unknown,2023-01-17,06:14:52
20381,ABM00000000000021507,SYNCID0000016882,1937.08,credit,False,CA,ON,VAUGHAN,2022-11-21,13:01:01
20382,ABM00000000000021508,SYNCID0000016882,23375.23,credit,False,CA,ON,TORONTO,2022-12-16,17:45:51
20383,ABM00000000000021509,SYNCID0000016882,2039.37,credit,False,CA,ON,HALTONHLLS,2022-11-03,09:49:13
20384,ABM00000000000021510,SYNCID0000016882,52402.54,credit,False,other,unknown,other,2022-11-15,12:14:52
20385,ABM00000000000021511,SYNCID0000016882,1058.62,debit,True,CA,ON,PETERBOROUGH,2022-11-18,17:21:28
20386,ABM00000000000021512,SYNCID0000016882,422.46,debit,True,CA,unknown,other,2023-01-12,19:35:40
20387,ABM00000000000021513,SYNCID0000016882,3530.50,credit,False,CA,SK,SASKATOON,2022-12-05,08:18:41
20388,ABM00000000000021514,SYNCID0000016882,210.77,debit,True,other,unknown,other,2022-11-22,09:23:01


In [182]:
abm[abm["customer_id"] == "SYNCID0000013414"]

,abm_id,customer_id,amount_cad,debit_credit,cash_indicator,country,province,city,transaction_date,transaction_time
15896,ABM00000000000016694,SYNCID0000013414,53.52,debit,True,unknown,unknown,unknown,2022-12-07,18:36:32
15897,ABM00000000000016695,SYNCID0000013414,66.73,debit,True,unknown,unknown,unknown,2023-01-13,12:40:26
15898,ABM00000000000016696,SYNCID0000013414,428.17,credit,False,CA,QC,STJEROME,2022-11-09,11:51:05
15899,ABM00000000000016697,SYNCID0000013414,1083.14,debit,True,CA,ON,NIAGARA FLS,2022-11-23,14:17:30
15900,ABM00000000000016698,SYNCID0000013414,860.22,debit,True,CA,NS,HALIFAX,2022-12-25,14:59:11
15901,ABM00000000000016699,SYNCID0000013414,1026.54,debit,True,CA,ON,STCATHARINES,2022-11-01,15:00:33
15902,ABM00000000000016700,SYNCID0000013414,66.65,debit,True,CA,ON,WOODBRIDGE,2022-12-04,15:14:33
15903,ABM00000000000016701,SYNCID0000013414,430.41,credit,True,CA,ON,TORONTO,2022-11-27,21:12:57
15904,ABM00000000000016702,SYNCID0000013414,1039.85,debit,True,CA,ON,NEPEAN,2023-01-24,16:43:26
15905,ABM00000000000016703,SYNCID0000013414,1070.11,debit,True,CA,BC,BURNABY,2022-12-02,11:32:11


In [183]:
abm_t1 = abm[(abm["cash_indicator"] == True) & (abm["amount_cad"] > 10000) & (abm["debit_credit"] == "credit")]
# large cash transaction

In [184]:
abm_ids = abm["customer_id"].unique()

In [185]:
abm_cust = abm_t1["abm_id"].unique()

In [186]:
def get_status(customer_id):
    if customer_id in abm_ids:
        if customer_id in abm_cust:
            return 2  # strange ones
        else:
            return 0  # fine customer in ABM
    else:
        return 1  # not in ABM

In [187]:
kyc['abm'] = kyc['customer_id'].apply(get_status)

In [188]:
kyc

,customer_id,country,province,city,industry_code,employee_count,sales,established_date,onboard_date,abm
0,SYNCID0000000000,CA,ON,NORTH YORK,7292,0.0,0.0,2019-06-22,2022-05-02,1
1,SYNCID0000000001,CA,ON,NORTH YORK,NaN,NaN,66446.0,2009-07-21,2009-08-21,1
2,SYNCID0000000002,CA,ON,KITCHENER,7721,2.0,NaN,2022-04-11,2022-09-14,1
3,SYNCID0000000004,CA,NaN,other,4565,1.0,328160.0,2020-11-16,2022-12-08,1
4,SYNCID0000000005,CA,BC,VICTORIA,7799,1.0,0.0,2022-01-21,2023-01-01,1
...,...,...,...,...,...,...,...,...,...,...
16250,SYNCID0000017178,CA,ON,TORONTO,7759,1.0,0.0,2017-09-22,2017-12-02,1
16251,SYNCID0000017179,CA,ON,NORTH YORK,7771,0.0,0.0,2013-06-22,2013-12-07,1
16252,SYNCID0000017180,CA,NaN,other,0211,NaN,1.0,1981-03-11,2017-11-30,1
16253,SYNCID0000017181,CA,ON,SCARBOROUGH,8661,20.0,4833395.0,2015-09-05,2015-09-14,1


# Card

In [189]:
card.head(10)

,card_trxn_id,customer_id,amount_cad,debit_credit,merchant_category,ecommerce_ind,country,province,city,transaction_date,transaction_time
0,CON00000000000000000,SYNCID0000000001,60.30,debit,other,False,CA,AB,CALGARY,2022-12-20,17:14:40
1,CON00000000000000001,SYNCID0000000001,1.72,debit,4121,True,NaN,NaN,other,2022-11-02,13:28:57
2,CON00000000000000002,SYNCID0000000001,15.97,debit,other,False,NaN,NaN,other,2022-11-29,10:55:08
3,CON00000000000000003,SYNCID0000000001,213.40,debit,5542,False,CA,NB,MONCTON,2022-11-21,09:15:48
4,CON00000000000000077,SYNCID0000000004,-137.42,credit,7399,False,CA,QC,VERDUN,2022-12-02,17:56:27
5,CON00000000000000078,SYNCID0000000004,182.91,debit,5541,False,NaN,NaN,other,2023-01-09,10:51:40
6,CON00000000000000079,SYNCID0000000004,162.50,debit,5541,False,NaN,NaN,NaN,2023-01-10,16:21:44
7,CON00000000000000080,SYNCID0000000004,35.04,debit,5411,False,NaN,NaN,other,2022-11-22,17:06:24
8,CON00000000000000081,SYNCID0000000004,43.02,debit,5734,False,CA,AB,NaN,2023-01-28,07:44:14
9,CON00000000000000082,SYNCID0000000004,72.91,debit,5814,False,NaN,NaN,other,2023-01-03,19:01:21


In [190]:
card_ids = card["customer_id"].unique()

In [191]:
card_t1 = card[(card["amount_cad"] > 10000) & card["country"].isnull()]

In [192]:
card_cust = card_t1["customer_id"].unique()

In [193]:
def get_status(customer_id):
    if customer_id in card_ids:
        if customer_id in card_cust:
            return 2  # strange ones
        else:
            return 0  # fine customer 
    else:
        return 1  # not in table

In [194]:
kyc['card'] = kyc['customer_id'].apply(get_status)

In [195]:
kyc

,customer_id,country,province,city,industry_code,employee_count,sales,established_date,onboard_date,abm,card
0,SYNCID0000000000,CA,ON,NORTH YORK,7292,0.0,0.0,2019-06-22,2022-05-02,1,1
1,SYNCID0000000001,CA,ON,NORTH YORK,NaN,NaN,66446.0,2009-07-21,2009-08-21,1,0
2,SYNCID0000000002,CA,ON,KITCHENER,7721,2.0,NaN,2022-04-11,2022-09-14,1,1
3,SYNCID0000000004,CA,NaN,other,4565,1.0,328160.0,2020-11-16,2022-12-08,1,0
4,SYNCID0000000005,CA,BC,VICTORIA,7799,1.0,0.0,2022-01-21,2023-01-01,1,1
...,...,...,...,...,...,...,...,...,...,...,...
16250,SYNCID0000017178,CA,ON,TORONTO,7759,1.0,0.0,2017-09-22,2017-12-02,1,1
16251,SYNCID0000017179,CA,ON,NORTH YORK,7771,0.0,0.0,2013-06-22,2013-12-07,1,1
16252,SYNCID0000017180,CA,NaN,other,0211,NaN,1.0,1981-03-11,2017-11-30,1,1
16253,SYNCID0000017181,CA,ON,SCARBOROUGH,8661,20.0,4833395.0,2015-09-05,2015-09-14,1,1


# cheque


### 1. An anomaly in the transaction amount > 150k
### 2. high frequency high-value transaction (> 8 times / month, amount > 10k)

In [196]:
cheque.head(10)
cheque_cust = set()

In [197]:
cheque_ids = cheque["customer_id"].unique()

In [198]:
# cheque type 1
cheque_t1 = cheque[cheque["amount_cad"] > 150000]
cheque_cust.update(cheque_t1["customer_id"].unique())

In [199]:
# cheque type 2
large_cheque = cheque[cheque['amount_cad'] > 10000]
customer_counts = large_cheque['customer_id'].value_counts()
large_cheque['transaction_date1'] = pd.to_datetime(large_cheque['transaction_date'])
large_cheque.set_index('transaction_date1', inplace=True)
transaction_frequency = large_cheque.groupby('customer_id').resample('M').size()


transaction_frequency = transaction_frequency.reset_index()
transaction_frequency.columns = ['customer_id', 'month', 'transaction_count']
high_frequency_customers = transaction_frequency[transaction_frequency['transaction_count'] > 8]

print(high_frequency_customers)


/var/folders/8b/cv3m4j_s1y11xcc8l3vn77080000gn/T/ipykernel_57839/906774303.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  large_cheque['transaction_date1'] = pd.to_datetime(large_cheque['transaction_date'])
/var/folders/8b/cv3m4j_s1y11xcc8l3vn77080000gn/T/ipykernel_57839/906774303.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  transaction_frequency = large_cheque.groupby('customer_id').resample('M').size()


           customer_id      month  transaction_count
23    SYNCID0000000047 2022-11-30                 12
25    SYNCID0000000047 2023-01-31                 12
166   SYNCID0000000319 2022-12-31                  9
203   SYNCID0000000386 2022-11-30                 13
204   SYNCID0000000386 2022-12-31                 18
...                ...        ...                ...
9642  SYNCID0000017095 2023-01-31                 13
9653  SYNCID0000017116 2022-11-30                 10
9675  SYNCID0000017137 2022-11-30                 42
9676  SYNCID0000017137 2022-12-31                 30
9677  SYNCID0000017137 2023-01-31                 19

[450 rows x 3 columns]


In [200]:
cheque_t2 = high_frequency_customers
cheque_cust.update(cheque_t2["customer_id"].unique()) 

In [201]:
def get_status(customer_id):
    if customer_id in cheque_ids:
        if customer_id in cheque_cust:
            return 2  # strange ones
        else:
            return 0  # fine customer 
    else:
        return 1  # not in table

In [202]:
kyc['cheque'] = kyc['customer_id'].apply(get_status)

In [203]:
kyc

,customer_id,country,province,city,industry_code,employee_count,sales,established_date,onboard_date,abm,card,cheque
0,SYNCID0000000000,CA,ON,NORTH YORK,7292,0.0,0.0,2019-06-22,2022-05-02,1,1,0
1,SYNCID0000000001,CA,ON,NORTH YORK,NaN,NaN,66446.0,2009-07-21,2009-08-21,1,0,1
2,SYNCID0000000002,CA,ON,KITCHENER,7721,2.0,NaN,2022-04-11,2022-09-14,1,1,0
3,SYNCID0000000004,CA,NaN,other,4565,1.0,328160.0,2020-11-16,2022-12-08,1,0,1
4,SYNCID0000000005,CA,BC,VICTORIA,7799,1.0,0.0,2022-01-21,2023-01-01,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
16250,SYNCID0000017178,CA,ON,TORONTO,7759,1.0,0.0,2017-09-22,2017-12-02,1,1,0
16251,SYNCID0000017179,CA,ON,NORTH YORK,7771,0.0,0.0,2013-06-22,2013-12-07,1,1,1
16252,SYNCID0000017180,CA,NaN,other,0211,NaN,1.0,1981-03-11,2017-11-30,1,1,1
16253,SYNCID0000017181,CA,ON,SCARBOROUGH,8661,20.0,4833395.0,2015-09-05,2015-09-14,1,1,1


# eft

### 1. An anomaly in the transaction amount > 100k
### 2. large transaction (>= 50k) record >= 10
### 3. large transaction (>= 50k) at night / weekend

In [204]:
eft.head(10)
eft_ids = eft["customer_id"].unique()

In [205]:
# strange customer
eft[eft["customer_id"] == "SYNCID0000004105"]

,eft_id,customer_id,amount_cad,debit_credit,transaction_date,transaction_time
122917,EFT00000000000126896,SYNCID0000004105,4560.48,debit,2023-01-10,17:08:26
122918,EFT00000000000126897,SYNCID0000004105,12.53,debit,2022-12-05,10:25:03
122919,EFT00000000000126898,SYNCID0000004105,3578.35,credit,2023-01-18,21:12:32
122920,EFT00000000000126899,SYNCID0000004105,192.57,debit,2022-11-04,11:01:08
122921,EFT00000000000126900,SYNCID0000004105,53553.93,credit,2022-11-02,21:11:38
...,...,...,...,...,...,...
125201,EFT00000000000129180,SYNCID0000004105,7974.53,debit,2022-12-19,21:31:09
125202,EFT00000000000129181,SYNCID0000004105,377.10,debit,2022-11-18,12:51:05
125203,EFT00000000000129182,SYNCID0000004105,937.20,debit,2023-01-11,21:27:54
125204,EFT00000000000129183,SYNCID0000004105,1761.81,credit,2023-01-09,16:41:55


In [206]:
# eft type 1
eft_t1 = eft[eft["amount_cad"] > 100000]

In [207]:
# eft type 2
eft_50k = eft[eft["amount_cad"] > 50000]

In [208]:
transaction_counts = eft_50k['customer_id'].value_counts()

high_transaction_customers = transaction_counts[transaction_counts > 10]
print(high_transaction_customers)
eft_t2 = high_transaction_customers

customer_id
SYNCID0000004105    65
SYNCID0000008526    56
SYNCID0000012928    55
SYNCID0000011351    47
SYNCID0000009036    39
                    ..
SYNCID0000007663    11
SYNCID0000008611    11
SYNCID0000004346    11
SYNCID0000016926    11
SYNCID0000015608    11
Name: count, Length: 91, dtype: int64


In [209]:
# eft type 3
eft['transaction_date1'] = pd.to_datetime(eft['transaction_date'])
eft['is_weekday'] = eft['transaction_date1'].apply(lambda x: x.weekday() < 5)

In [210]:
from datetime import datetime, time

eft["transaction_time1"] = pd.to_datetime(eft['transaction_time'], format='%H:%M:%S').dt.time
start_time = time(23, 0)  # 23:00
end_time = time(6, 0)  # 06:00 
eft['rest_time'] = eft["transaction_time1"].apply(lambda x: x >= start_time or x < end_time)

In [211]:
eft_50k = eft[eft["amount_cad"] > 50000]
eft_t3 = eft_50k[(eft_50k["rest_time"] == True) | (eft_50k["is_weekday"] == False)]

In [212]:
eft_t2

customer_id
SYNCID0000004105    65
SYNCID0000008526    56
SYNCID0000012928    55
SYNCID0000011351    47
SYNCID0000009036    39
                    ..
SYNCID0000007663    11
SYNCID0000008611    11
SYNCID0000004346    11
SYNCID0000016926    11
SYNCID0000015608    11
Name: count, Length: 91, dtype: int64

In [213]:
eft_t2

customer_id
SYNCID0000004105    65
SYNCID0000008526    56
SYNCID0000012928    55
SYNCID0000011351    47
SYNCID0000009036    39
                    ..
SYNCID0000007663    11
SYNCID0000008611    11
SYNCID0000004346    11
SYNCID0000016926    11
SYNCID0000015608    11
Name: count, Length: 91, dtype: int64

In [214]:
eft_cust = set()

eft_cust.update(eft_t1["customer_id"].unique())
eft_cust.update(eft_t2.index.to_list())
eft_cust.update(eft_t3["customer_id"].unique())

In [215]:
def get_status(customer_id):
    if customer_id in eft_ids:
        if customer_id in eft_cust:
            return 2  # strange ones
        else:
            return 0  # fine customer 
    else:
        return 1  # not in table

In [216]:
kyc['eft'] = kyc['customer_id'].apply(get_status)

In [217]:
kyc

,customer_id,country,province,city,industry_code,employee_count,sales,established_date,onboard_date,abm,card,cheque,eft
0,SYNCID0000000000,CA,ON,NORTH YORK,7292,0.0,0.0,2019-06-22,2022-05-02,1,1,0,1
1,SYNCID0000000001,CA,ON,NORTH YORK,NaN,NaN,66446.0,2009-07-21,2009-08-21,1,0,1,1
2,SYNCID0000000002,CA,ON,KITCHENER,7721,2.0,NaN,2022-04-11,2022-09-14,1,1,0,1
3,SYNCID0000000004,CA,NaN,other,4565,1.0,328160.0,2020-11-16,2022-12-08,1,0,1,1
4,SYNCID0000000005,CA,BC,VICTORIA,7799,1.0,0.0,2022-01-21,2023-01-01,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16250,SYNCID0000017178,CA,ON,TORONTO,7759,1.0,0.0,2017-09-22,2017-12-02,1,1,0,0
16251,SYNCID0000017179,CA,ON,NORTH YORK,7771,0.0,0.0,2013-06-22,2013-12-07,1,1,1,2
16252,SYNCID0000017180,CA,NaN,other,0211,NaN,1.0,1981-03-11,2017-11-30,1,1,1,1
16253,SYNCID0000017181,CA,ON,SCARBOROUGH,8661,20.0,4833395.0,2015-09-05,2015-09-14,1,1,1,0


# emt

### 1. large transaction (>= 10k) at night / weekend
### 2. high frequency mid-value transaction (> 30 times/month, amount > 1k)
### 3. high frequenct small-value transaction(> 100 times/month. amount < 1k)

In [218]:
emt.head(10)
emt_ids = emt["customer_id"].unique()

In [219]:
# emt type 1
emt['transaction_date1'] = pd.to_datetime(emt['transaction_date'])
emt['is_weekday'] = emt['transaction_date1'].apply(lambda x: x.weekday() < 5)


emt["transaction_time1"] = pd.to_datetime(emt['transaction_time'], format='%H:%M:%S').dt.time
start_time = time(23, 0)  # 23:00
end_time = time(6, 0)  # 06:00 
emt['rest_time'] = emt["transaction_time1"].apply(lambda x: x >= start_time or x < end_time)

In [220]:
emt_50k = emt[emt["amount_cad"] >= 10000]
emt_t1 = emt_50k[(emt_50k["is_weekday"] == False) | (emt_50k["rest_time"] == True)]

In [221]:
# emt type 2
mid_emt = emt[emt['amount_cad'] > 1000]
customer_counts = mid_emt['customer_id'].value_counts()

mid_emt.set_index('transaction_date1', inplace=True)
transaction_frequency = mid_emt.groupby('customer_id').resample('M').size()

transaction_frequency = transaction_frequency.reset_index()
transaction_frequency.columns = ['customer_id', 'month', 'transaction_count']
high_frequency_customers = transaction_frequency[transaction_frequency['transaction_count'] > 30]

print(high_frequency_customers)
emt_t2 = high_frequency_customers

/var/folders/8b/cv3m4j_s1y11xcc8l3vn77080000gn/T/ipykernel_57839/3585897343.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  transaction_frequency = mid_emt.groupby('customer_id').resample('M').size()


            customer_id      month  transaction_count
499    SYNCID0000000619 2022-11-30                 98
500    SYNCID0000000619 2022-12-31                118
501    SYNCID0000000619 2023-01-31                 33
524    SYNCID0000000648 2022-12-31                 35
561    SYNCID0000000692 2022-11-30                 36
...                 ...        ...                ...
12082  SYNCID0000015019 2022-11-30                 32
12083  SYNCID0000015019 2022-12-31                 34
12976  SYNCID0000016130 2022-11-30                 42
12977  SYNCID0000016130 2022-12-31                 38
13330  SYNCID0000016536 2022-11-30                 41

[76 rows x 3 columns]


In [222]:
# emt type 3
small_emt = emt[emt['amount_cad'] <= 1000]
customer_counts = small_emt['customer_id'].value_counts()

small_emt.set_index('transaction_date1', inplace=True)
transaction_frequency = small_emt.groupby('customer_id').resample('M').size()

transaction_frequency = transaction_frequency.reset_index()
transaction_frequency.columns = ['customer_id', 'month', 'transaction_count']
high_frequency_customers = transaction_frequency[transaction_frequency['transaction_count'] > 100]

print(high_frequency_customers)
emt_t3 = high_frequency_customers

/var/folders/8b/cv3m4j_s1y11xcc8l3vn77080000gn/T/ipykernel_57839/226117616.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  transaction_frequency = small_emt.groupby('customer_id').resample('M').size()


            customer_id      month  transaction_count
612    SYNCID0000000619 2022-11-30                201
613    SYNCID0000000619 2022-12-31                207
687    SYNCID0000000692 2022-11-30                159
688    SYNCID0000000692 2022-12-31                179
881    SYNCID0000000871 2022-11-30                102
1326   SYNCID0000001356 2022-11-30                134
1327   SYNCID0000001356 2022-12-31                117
1386   SYNCID0000001413 2022-11-30               1043
1387   SYNCID0000001413 2022-12-31               1629
1388   SYNCID0000001413 2023-01-31                139
2466   SYNCID0000002454 2022-11-30                101
3270   SYNCID0000003294 2022-11-30                487
3271   SYNCID0000003294 2022-12-31                464
3689   SYNCID0000003626 2022-11-30                132
3690   SYNCID0000003626 2022-12-31                138
3754   SYNCID0000003681 2022-11-30                144
3755   SYNCID0000003681 2022-12-31                147
3874   SYNCID0000003802 2022

In [223]:
emt_cust = set()

emt_cust.update(emt_t1["customer_id"].unique())
emt_cust.update(emt_t2["customer_id"].unique())
emt_cust.update(emt_t3["customer_id"].unique())

In [224]:
def get_status(customer_id):
    if customer_id in emt_ids:
        if customer_id in emt_cust:
            return 2  # strange ones
        else:
            return 0  # fine customer 
    else:
        return 1  # not in table

In [225]:
kyc['emt'] = kyc['customer_id'].apply(get_status)

In [226]:
kyc

,customer_id,country,province,city,industry_code,employee_count,sales,established_date,onboard_date,abm,card,cheque,eft,emt
0,SYNCID0000000000,CA,ON,NORTH YORK,7292,0.0,0.0,2019-06-22,2022-05-02,1,1,0,1,1
1,SYNCID0000000001,CA,ON,NORTH YORK,NaN,NaN,66446.0,2009-07-21,2009-08-21,1,0,1,1,1
2,SYNCID0000000002,CA,ON,KITCHENER,7721,2.0,NaN,2022-04-11,2022-09-14,1,1,0,1,1
3,SYNCID0000000004,CA,NaN,other,4565,1.0,328160.0,2020-11-16,2022-12-08,1,0,1,1,1
4,SYNCID0000000005,CA,BC,VICTORIA,7799,1.0,0.0,2022-01-21,2023-01-01,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16250,SYNCID0000017178,CA,ON,TORONTO,7759,1.0,0.0,2017-09-22,2017-12-02,1,1,0,0,1
16251,SYNCID0000017179,CA,ON,NORTH YORK,7771,0.0,0.0,2013-06-22,2013-12-07,1,1,1,2,1
16252,SYNCID0000017180,CA,NaN,other,0211,NaN,1.0,1981-03-11,2017-11-30,1,1,1,1,0
16253,SYNCID0000017181,CA,ON,SCARBOROUGH,8661,20.0,4833395.0,2015-09-05,2015-09-14,1,1,1,0,1


# Wire

# large amount > 100k

In [227]:
wire.head(10)
wire_ids = wire["customer_id"].unique()

In [228]:
wire_t1 = wire[wire["amount_cad"] > 100000]

In [229]:
wire_cust = wire_t1["customer_id"].unique()

In [230]:
def get_status(customer_id):
    if customer_id in wire_ids:
        if customer_id in wire_cust:
            return 2  # strange ones
        else:
            return 0  # fine customer 
    else:
        return 1  # not in table

In [231]:
kyc['wire'] = kyc['customer_id'].apply(get_status)

In [232]:
kyc

,customer_id,country,province,city,industry_code,employee_count,sales,established_date,onboard_date,abm,card,cheque,eft,emt,wire
0,SYNCID0000000000,CA,ON,NORTH YORK,7292,0.0,0.0,2019-06-22,2022-05-02,1,1,0,1,1,0
1,SYNCID0000000001,CA,ON,NORTH YORK,NaN,NaN,66446.0,2009-07-21,2009-08-21,1,0,1,1,1,1
2,SYNCID0000000002,CA,ON,KITCHENER,7721,2.0,NaN,2022-04-11,2022-09-14,1,1,0,1,1,1
3,SYNCID0000000004,CA,NaN,other,4565,1.0,328160.0,2020-11-16,2022-12-08,1,0,1,1,1,1
4,SYNCID0000000005,CA,BC,VICTORIA,7799,1.0,0.0,2022-01-21,2023-01-01,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16250,SYNCID0000017178,CA,ON,TORONTO,7759,1.0,0.0,2017-09-22,2017-12-02,1,1,0,0,1,1
16251,SYNCID0000017179,CA,ON,NORTH YORK,7771,0.0,0.0,2013-06-22,2013-12-07,1,1,1,2,1,1
16252,SYNCID0000017180,CA,NaN,other,0211,NaN,1.0,1981-03-11,2017-11-30,1,1,1,1,0,1
16253,SYNCID0000017181,CA,ON,SCARBOROUGH,8661,20.0,4833395.0,2015-09-05,2015-09-14,1,1,1,0,1,1


# Know Your Customer

In [233]:
kyc.head(10)

###### INTRO TASK - Using 10 features to perform KNN, so we will add 10 additional columns on kyc table, and use that to perform classification

# 1. Business History = onboard_date - estabilished_date

# 2. Average sales/person = sales / employee_count

# 3. (Olivia) Financial transparency -- total transaction amount / total transaction times, in all methods 

# 4. Industry Code 

# 5. Consistency between business location and place of registration
   #-- Using "kyc" LEFT JOIN "card" for example, look at the rate of DIFF(city in kyc, city in abm)
   
# (Need discussion on exaxt rules for each transaction type) 6.7.8.9.10. Unusual customer in 
#     EFT & Wire 
#     Card
#     EMT
#     ABM 
#     Cheque (1 -- Yes, 0 --- No, unknown)

# 11. (Olivia) credit / debit rate -- Too low, too high => unnormal， alternate C/D (complex)

# Question: Do you want to consider the risk of company location? 

# ==================================================================================================================================

###### TASK 3 -- Applying INTRO TASK, we identify 100 customers' property (0, 1) and offer labels, 
# leaving others unlabeled, perform self-supervised learning.


,customer_id,country,province,city,industry_code,employee_count,sales,established_date,onboard_date,abm,card,cheque,eft,emt,wire
0,SYNCID0000000000,CA,ON,NORTH YORK,7292,0.0,0.0,2019-06-22,2022-05-02,1,1,0,1,1,0
1,SYNCID0000000001,CA,ON,NORTH YORK,NaN,NaN,66446.0,2009-07-21,2009-08-21,1,0,1,1,1,1
2,SYNCID0000000002,CA,ON,KITCHENER,7721,2.0,NaN,2022-04-11,2022-09-14,1,1,0,1,1,1
3,SYNCID0000000004,CA,NaN,other,4565,1.0,328160.0,2020-11-16,2022-12-08,1,0,1,1,1,1
4,SYNCID0000000005,CA,BC,VICTORIA,7799,1.0,0.0,2022-01-21,2023-01-01,1,1,1,0,1,1
5,SYNCID0000000006,CA,BC,other,7761,0.0,0.0,2009-11-05,NaN,1,1,0,1,1,1
6,SYNCID0000000007,CA,BC,PENTICTON,7292,1.0,33642.0,1987-05-29,2018-09-24,1,0,0,0,0,1
7,SYNCID0000000008,CA,NaN,other,9659,0.0,0.0,1992-10-07,1992-01-08,1,1,0,1,1,1
8,SYNCID0000000009,CA,QC,MONTREAL,8653,1.0,73595.0,2012-07-01,2019-01-21,1,0,0,0,1,1
9,SYNCID0000000010,CA,ON,TORONTO,7511,0.0,NaN,2007-08-27,2008-04-26,1,1,0,0,1,1


In [234]:
# 2. Average sales/person = sales / employee_count
kyc["sale_employee_rate"] = (kyc["sales"] / kyc["employee_count"]).fillna("unknown")


In [235]:
# 1. Business History = onboard_date - estabilished_date
kyc["established_date"] = pd.to_datetime(kyc["established_date"])
kyc["onboard_date"] = pd.to_datetime(kyc["onboard_date"])
kyc["business_history"] = (kyc["onboard_date"] - kyc["established_date"]).dt.days
kyc["business_history"] = kyc["business_history"].apply(lambda x: x if x >= 0 else "unreasonable" if x < 0 else "unknown")

In [236]:
kyc

,customer_id,country,province,city,industry_code,employee_count,sales,established_date,onboard_date,abm,card,cheque,eft,emt,wire,sale_employee_rate,business_history
0,SYNCID0000000000,CA,ON,NORTH YORK,7292,0.0,0.0,2019-06-22,2022-05-02,1,1,0,1,1,0,unknown,1045.0
1,SYNCID0000000001,CA,ON,NORTH YORK,NaN,NaN,66446.0,2009-07-21,2009-08-21,1,0,1,1,1,1,unknown,31.0
2,SYNCID0000000002,CA,ON,KITCHENER,7721,2.0,NaN,2022-04-11,2022-09-14,1,1,0,1,1,1,unknown,156.0
3,SYNCID0000000004,CA,NaN,other,4565,1.0,328160.0,2020-11-16,2022-12-08,1,0,1,1,1,1,328160.0,752.0
4,SYNCID0000000005,CA,BC,VICTORIA,7799,1.0,0.0,2022-01-21,2023-01-01,1,1,1,0,1,1,0.0,345.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16250,SYNCID0000017178,CA,ON,TORONTO,7759,1.0,0.0,2017-09-22,2017-12-02,1,1,0,0,1,1,0.0,71.0
16251,SYNCID0000017179,CA,ON,NORTH YORK,7771,0.0,0.0,2013-06-22,2013-12-07,1,1,1,2,1,1,unknown,168.0
16252,SYNCID0000017180,CA,NaN,other,0211,NaN,1.0,1981-03-11,2017-11-30,1,1,1,1,0,1,unknown,13413.0
16253,SYNCID0000017181,CA,ON,SCARBOROUGH,8661,20.0,4833395.0,2015-09-05,2015-09-14,1,1,1,0,1,1,241669.75,9.0


In [237]:
kyc_abm["country_x"]

0        CA
1        CA
2        CA
3        CA
4        CA
         ..
20652    CA
20653    CA
20654    CA
20655    CA
20656    CA
Name: country_x, Length: 20657, dtype: object

In [238]:
# 5. Consistency between business location and place of registration
# ABM & CARD
kyc_abm = kyc.merge(abm, on="customer_id", how="inner")

kyc_abm['country_match'] = (kyc_abm['country_x'] == kyc_abm['country_y']).astype(int)

summary = kyc_abm.groupby('customer_id').agg(
    total_records=('customer_id', 'size'),        
    matches=('country_match', 'sum')
    ).reset_index()


In [239]:
summary.head(70)

,customer_id,total_records,matches
0,SYNCID0000000014,1,1
1,SYNCID0000000034,5,5
2,SYNCID0000000038,1,1
3,SYNCID0000000046,29,20
4,SYNCID0000000051,3,3
...,...,...,...
65,SYNCID0000000313,1,1
66,SYNCID0000000329,1,1
67,SYNCID0000000337,1,1
68,SYNCID0000000340,1,1


In [240]:

summary['proportion_matches'] = summary['matches'] / summary['total_records']
print(summary)

           customer_id  total_records  matches  proportion_matches
0     SYNCID0000000014              1        1            1.000000
1     SYNCID0000000034              5        5            1.000000
2     SYNCID0000000038              1        1            1.000000
3     SYNCID0000000046             29       20            0.689655
4     SYNCID0000000051              3        3            1.000000
...                ...            ...      ...                 ...
3607  SYNCID0000017163              1        0            0.000000
3608  SYNCID0000017168              9        7            0.777778
3609  SYNCID0000017172              1        1            1.000000
3610  SYNCID0000017174             18       15            0.833333
3611  SYNCID0000017175              6        3            0.500000

[3612 rows x 4 columns]


In [241]:
kyc_abm.head(10)

,customer_id,country_x,province_x,city_x,industry_code,employee_count,sales,established_date,onboard_date,abm,...,abm_id,amount_cad,debit_credit,cash_indicator,country_y,province_y,city_y,transaction_date,transaction_time,country_match
0,SYNCID0000000014,CA,ON,MARKHAM,4275,0.0,NaN,2016-09-14,NaT,0,...,ABM00000000000000006,25.41,credit,False,CA,SK,REGINA,2022-11-16,17:37:41,1
1,SYNCID0000000034,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,0,...,ABM00000000000000008,238.17,debit,True,CA,ON,TORONTO,2022-11-18,10:22:59,1
2,SYNCID0000000034,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,0,...,ABM00000000000000009,1655.43,credit,False,CA,unknown,other,2022-12-29,11:56:08,1
3,SYNCID0000000034,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,0,...,ABM00000000000000010,620.69,credit,True,CA,unknown,other,2023-01-22,16:48:12,1
4,SYNCID0000000034,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,0,...,ABM00000000000000011,323.70,debit,True,CA,unknown,other,2022-11-14,13:24:45,1
5,SYNCID0000000034,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,0,...,ABM00000000000000012,6876.76,credit,False,CA,ON,VAUGHAN,2022-12-11,11:55:50,1
6,SYNCID0000000038,CA,AB,EDMONTON,8652,0.0,0.0,2019-06-16,2019-08-07,0,...,ABM00000000000000013,104.55,debit,True,CA,unknown,other,2022-12-28,18:07:36,1
7,SYNCID0000000046,CA,NaN,other,4561,7.0,419221.0,2021-07-18,2022-07-31,0,...,ABM00000000000000014,111.91,debit,True,CA,AB,EDMONTON,2022-11-14,22:58:02,1
8,SYNCID0000000046,CA,NaN,other,4561,7.0,419221.0,2021-07-18,2022-07-31,0,...,ABM00000000000000015,193.76,debit,True,unknown,unknown,unknown,2022-11-26,10:50:41,0
9,SYNCID0000000046,CA,NaN,other,4561,7.0,419221.0,2021-07-18,2022-07-31,0,...,ABM00000000000000016,217.00,debit,True,unknown,unknown,unknown,2023-01-06,10:26:31,0


In [242]:

kyc_card = kyc.merge(card, on="customer_id", how="inner")

kyc_card['country_match'] = (kyc_card['country_x'] == kyc_card['country_y']).astype(int)

summary2 = kyc_card.groupby('customer_id').agg(
    total_records=('customer_id', 'size'),        
    matches=('country_match', 'sum')
    ).reset_index()

summary2

,customer_id,total_records,matches
0,SYNCID0000000001,4,2
1,SYNCID0000000004,44,15
2,SYNCID0000000007,16,7
3,SYNCID0000000009,32,16
4,SYNCID0000000011,9,4
...,...,...,...
9784,SYNCID0000017171,6,2
9785,SYNCID0000017172,6,3
9786,SYNCID0000017174,16,8
9787,SYNCID0000017175,26,14


In [243]:
summary2['proportion_matches2'] = summary2['matches'] / summary2['total_records']

# Result: A DataFrame with the desired proportions
print(summary2)

           customer_id  total_records  matches  proportion_matches2
0     SYNCID0000000001              4        2             0.500000
1     SYNCID0000000004             44       15             0.340909
2     SYNCID0000000007             16        7             0.437500
3     SYNCID0000000009             32       16             0.500000
4     SYNCID0000000011              9        4             0.444444
...                ...            ...      ...                  ...
9784  SYNCID0000017171              6        2             0.333333
9785  SYNCID0000017172              6        3             0.500000
9786  SYNCID0000017174             16        8             0.500000
9787  SYNCID0000017175             26       14             0.538462
9788  SYNCID0000017177              4        1             0.250000

[9789 rows x 4 columns]


In [244]:
kyc_new = kyc.merge(summary, on="customer_id", how="left")
kyc_new = kyc_new.drop(columns=["total_records", "matches"])
kyc_new["proportion_matches"] = kyc_new["proportion_matches"].fillna(1) #no abm record

In [245]:
kyc_new = kyc_new.merge(summary2, on="customer_id", how="left")
kyc_new = kyc_new.drop(columns=["total_records", "matches"])
kyc_new["proportion_matches2"] = kyc_new["proportion_matches2"].fillna(1) # "no card record"

In [246]:
kyc_new

,customer_id,country,province,city,industry_code,employee_count,sales,established_date,onboard_date,abm,card,cheque,eft,emt,wire,sale_employee_rate,business_history,proportion_matches,proportion_matches2
0,SYNCID0000000000,CA,ON,NORTH YORK,7292,0.0,0.0,2019-06-22,2022-05-02,1,1,0,1,1,0,unknown,1045.0,1.0,1.000000
1,SYNCID0000000001,CA,ON,NORTH YORK,NaN,NaN,66446.0,2009-07-21,2009-08-21,1,0,1,1,1,1,unknown,31.0,1.0,0.500000
2,SYNCID0000000002,CA,ON,KITCHENER,7721,2.0,NaN,2022-04-11,2022-09-14,1,1,0,1,1,1,unknown,156.0,1.0,1.000000
3,SYNCID0000000004,CA,NaN,other,4565,1.0,328160.0,2020-11-16,2022-12-08,1,0,1,1,1,1,328160.0,752.0,1.0,0.340909
4,SYNCID0000000005,CA,BC,VICTORIA,7799,1.0,0.0,2022-01-21,2023-01-01,1,1,1,0,1,1,0.0,345.0,1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16250,SYNCID0000017178,CA,ON,TORONTO,7759,1.0,0.0,2017-09-22,2017-12-02,1,1,0,0,1,1,0.0,71.0,1.0,1.000000
16251,SYNCID0000017179,CA,ON,NORTH YORK,7771,0.0,0.0,2013-06-22,2013-12-07,1,1,1,2,1,1,unknown,168.0,1.0,1.000000
16252,SYNCID0000017180,CA,NaN,other,0211,NaN,1.0,1981-03-11,2017-11-30,1,1,1,1,0,1,unknown,13413.0,1.0,1.000000
16253,SYNCID0000017181,CA,ON,SCARBOROUGH,8661,20.0,4833395.0,2015-09-05,2015-09-14,1,1,1,0,1,1,241669.75,9.0,1.0,1.000000


In [247]:
kyc_new.head(10)

,customer_id,country,province,city,industry_code,employee_count,sales,established_date,onboard_date,abm,card,cheque,eft,emt,wire,sale_employee_rate,business_history,proportion_matches,proportion_matches2
0,SYNCID0000000000,CA,ON,NORTH YORK,7292,0.0,0.0,2019-06-22,2022-05-02,1,1,0,1,1,0,unknown,1045.0,1.0,1.000000
1,SYNCID0000000001,CA,ON,NORTH YORK,NaN,NaN,66446.0,2009-07-21,2009-08-21,1,0,1,1,1,1,unknown,31.0,1.0,0.500000
2,SYNCID0000000002,CA,ON,KITCHENER,7721,2.0,NaN,2022-04-11,2022-09-14,1,1,0,1,1,1,unknown,156.0,1.0,1.000000
3,SYNCID0000000004,CA,NaN,other,4565,1.0,328160.0,2020-11-16,2022-12-08,1,0,1,1,1,1,328160.0,752.0,1.0,0.340909
4,SYNCID0000000005,CA,BC,VICTORIA,7799,1.0,0.0,2022-01-21,2023-01-01,1,1,1,0,1,1,0.0,345.0,1.0,1.000000
5,SYNCID0000000006,CA,BC,other,7761,0.0,0.0,2009-11-05,NaT,1,1,0,1,1,1,unknown,unknown,1.0,1.000000
6,SYNCID0000000007,CA,BC,PENTICTON,7292,1.0,33642.0,1987-05-29,2018-09-24,1,0,0,0,0,1,33642.0,11441.0,1.0,0.437500
7,SYNCID0000000008,CA,NaN,other,9659,0.0,0.0,1992-10-07,1992-01-08,1,1,0,1,1,1,unknown,unreasonable,1.0,1.000000
8,SYNCID0000000009,CA,QC,MONTREAL,8653,1.0,73595.0,2012-07-01,2019-01-21,1,0,0,0,1,1,73595.0,2395.0,1.0,0.500000
9,SYNCID0000000010,CA,ON,TORONTO,7511,0.0,NaN,2007-08-27,2008-04-26,1,1,0,0,1,1,unknown,243.0,1.0,1.000000


## Additional Feature Engineering

In [248]:
#transaction total: total amount (sum of debits and credits; currently negative credits in cards.csv were not accounted for)
#transaction_count_debit: total number of debit transactions
#transaction_count_credit: total number of credit transactions
#transaction_average: average amount per transaction (negative credits in cards.cav not accounted for)
#debit_ratio = the total $ amount in debit transactions divided by total $ of transactions

aggregate_features = all_transactions.groupby('customer_id').agg(
    transaction_total = ('amount_cad', lambda x: x.sum()),
    transaction_count_debit = ('amount_cad', lambda x: x[all_transactions.loc[x.index, 'debit_credit'] == 'debit'].count()),
    transaction_count_credit = ('amount_cad', lambda x: x[all_transactions.loc[x.index, 'debit_credit'] == 'debit'].count()),
    transaction_average=('amount_cad', lambda x: x.sum() / x.count()),
    debit_ratio=('amount_cad', lambda x: x[all_transactions.loc[x.index, 'debit_credit'] == 'debit'].sum() / x.sum() if x.sum() > 0 else 0)
).reset_index()
aggregate_features

,customer_id,transaction_total,transaction_count_debit,transaction_count_credit,transaction_average,debit_ratio
0,SYNCID0000000000,6731.31,2,2,2243.770000,0.999996
1,SYNCID0000000001,291.39,4,4,72.847500,1.000000
2,SYNCID0000000002,202826.56,14,14,7512.094815,0.521647
3,SYNCID0000000004,17086.96,41,41,388.340000,0.440928
4,SYNCID0000000005,165250.47,15,15,3305.009400,0.759172
...,...,...,...,...,...,...
16221,SYNCID0000017178,135288.20,14,14,6442.295238,0.650150
16222,SYNCID0000017179,1527518.35,199,199,5494.670324,0.531026
16223,SYNCID0000017180,2663.13,5,5,177.542000,0.409113
16224,SYNCID0000017181,60459.54,11,11,1832.107273,0.191500


In [249]:
#add to the kyc table
kyc_new = kyc_new.merge(aggregate_features, on='customer_id', how='left')
kyc_new.head()

,customer_id,country,province,city,industry_code,employee_count,sales,established_date,onboard_date,abm,...,wire,sale_employee_rate,business_history,proportion_matches,proportion_matches2,transaction_total,transaction_count_debit,transaction_count_credit,transaction_average,debit_ratio
0,SYNCID0000000000,CA,ON,NORTH YORK,7292,0.0,0.0,2019-06-22,2022-05-02,1,...,0,unknown,1045.0,1.0,1.000000,6731.31,2.0,2.0,2243.770000,0.999996
1,SYNCID0000000001,CA,ON,NORTH YORK,NaN,NaN,66446.0,2009-07-21,2009-08-21,1,...,1,unknown,31.0,1.0,0.500000,291.39,4.0,4.0,72.847500,1.000000
2,SYNCID0000000002,CA,ON,KITCHENER,7721,2.0,NaN,2022-04-11,2022-09-14,1,...,1,unknown,156.0,1.0,1.000000,202826.56,14.0,14.0,7512.094815,0.521647
3,SYNCID0000000004,CA,NaN,other,4565,1.0,328160.0,2020-11-16,2022-12-08,1,...,1,328160.0,752.0,1.0,0.340909,17086.96,41.0,41.0,388.340000,0.440928
4,SYNCID0000000005,CA,BC,VICTORIA,7799,1.0,0.0,2022-01-21,2023-01-01,1,...,1,0.0,345.0,1.0,1.000000,165250.47,15.0,15.0,3305.009400,0.759172


In [250]:
#total $ amount from abm transactions that was in cash
cash_total = abm.loc[abm['cash_indicator'] == True].groupby('customer_id').agg(cash_total=('amount_cad', lambda x: x.abs().sum())).reset_index()

In [251]:
kyc_new = kyc_new.merge(cash_total, on='customer_id', how='left')
#create new column called cash_ratio (how much of their transactions are cash-baased)
kyc_new['cash_ratio'] = kyc_new['cash_total']/kyc_new['transaction_total']

In [252]:
#the NaNs just mean that there were no cash transactions
kyc_new['cash_ratio'] = kyc_new['cash_ratio'].fillna(0)
kyc_new['cash_total'] = kyc_new['cash_total'].fillna(0)

# Industry Codes

In [253]:
# see kyc_industry_codes.csv and major_group_classification.csv for definitions
#reduces the number of industries from 253 to 55 byt placing them into broader categories

In [254]:
kyc_new['industry_code_class'] = kyc_new['industry_code'].apply(
    lambda x: 'other' if x == 'other' else str(x)[:2]
)

In [255]:
kyc_new['industry_code_class'].unique().shape

(55,)

In [256]:
# uncomment for visualization
#import seaborn as sns
#industry_class_label = pd.read_csv('csv_files/major_group_classification.csv',dtype='object')
#industry_class_codes = pd.DataFrame(kyc_new['industry_code_class'])
#industry_class_codes.rename(columns={'industry_code_class': 'major_group'}, inplace=True)
#industry_class_df = industry_class_codes.merge(industry_class_label, on='major_group', how='left')

#fig=plt.figure(figsize=(8,10))
#sns.countplot(y=industry_class_df['name'])

'''

CHAT GPT ANSWER

1. Silhouette Score
What it measures: This score evaluates how similar an object is to its own cluster compared to other clusters. The silhouette score ranges from -1 to 1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters.
How to use: Compute the silhouette score for each sample, possibly using library functions from software like Python’s scikit-learn, and then average these values to obtain a final score.
2. Davies-Bouldin Index
What it measures: This index is intended to identify clusters that are well-separated and compact. A lower Davies-Bouldin index relates to a model with better separation between the clusters.
How to use: This index is calculable directly from features like the distance between centroids and the dispersion of points within each cluster. Libraries like scikit-learn can compute this index directly.
3. Calinski-Harabasz Index
What it measures: Also known as the Variance Ratio Criterion, this index measures the ratio of the sum of between-clusters dispersion to within-cluster dispersion. Higher values generally indicate better clustering.
How to use: This can be calculated using library functions and does not require any labels, relying instead on distances and variances that are intrinsic to the data.
4. Clustering Tendency
What it measures: Before even applying a clustering algorithm, it's important to assess whether the data tends to cluster at all.
How to use: Use the Hopkins statistic to determine the likelihood that the data has clusters rather than being uniformly distributed.
5. Visual Assessment
What it measures: Sometimes, the best way to understand the effectiveness of your clustering is by visualizing the clusters using techniques like PCA (Principal Component Analysis) or t-SNE (t-Distributed Stochastic Neighbor Embedding) for dimensionality reduction.
How to use: Plot the clusters after dimension reduction to see if they form distinct groups.
6. Stability Measures
What it measures: If you perturb the dataset slightly by adding noise or removing small subsets of data and then re-run the clustering, stable clusters should not change much.
How to use: Compare the results of clustering on the original data with those from perturbed versions to check consistency.
7. External Validation (if any external data available)
What it measures: If you have any external data that can serve as a rough proxy for cluster labels, you can use it to validate the clustering indirectly.
How to use: Apply some form of labeling or tagging based on external data, then use traditional classification metrics as a rough gauge.

'''